In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../../"))

from autocog.pm.sta.backend import Backend
from autocog.pm.sta.syntax import Syntax
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
sources = ["""
prompt mmlu_main {
    is {
        topic is text<20>;
        question is text<50>;
        choices[4] is {
            value is text<40>;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        _ as "You are answering a multiple choice questionnaire.";
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}""","""
format bool {
    is enum("true","false");
    annotate "a bolean value is either true or false";
}

format sentence {
    argument length=50;
    argument language="english";
    is text<length=$length>;
    annotate f"a grammatically correct {language} sentence of at most {length} tokens";
}

format response {
    is repeat(.choices.value);
    annotate f"repeat verbatim the value of the correct choice";
}

struct a_choice {
    is {
        value   is sentence<40>;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text<20>;
        question   is sentence;
        choices[4] is a_choice;
        answer     is response;
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        _ as "You are answering a multiple choice questionnaire.";
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
"""]
syntax = Syntax(header_pre_post=('[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>','[/INST]'))

In [4]:
for source in sources:
    backend = Backend(source=source)
    for (name, prompt) in backend.program.prompts.items():
        print(f"--- {name} ---")
        print(prompt.header())
        fta = syntax.fta(prompt)
        # for (a,act) in fta.actions.items():
        #     print(f"{a}: ")
        dotstr = fta.toGraphViz(label_with_uid=False)
        print(dotstr)
        display(wrap_graphviz(dotstr))
    print("====================")
    # print(compiler.program.prompts['mmlu_main'].fields[1].format.desc)
    # wrap_graphviz(compiler.program.toGraphViz())

--- mmlu_main ---
You are answering a multiple choice questionnaire.
You are using the folowing syntax:
```
start:
> topic(text(20)): the general category from which the question was taken
> question(text(50)): the question that you have to answer
> choices[4]: you judge whether each choice is correct or not
> > value(text(40)): the value of the choice
> > correct(enum("yes","no")): you decide whether this choice is correct or not
> answer(repeat(choices.value)): you repeat the value of the choice that best answer the question
```
n_header [label="[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>You are answering a multiple choice questionnaire.\nYou are using the following syntax:\n```\nstart:\n> topic(text(20)): the general category from which the question was taken\n> question(text(50)): the question that you have to answer\n> choices[4]: you judge whether each choice is correct or not\n> > value(text(40)): the value of the ch

--- mmlu_main ---
You are answering a multiple choice questionnaire.
You are using the folowing syntax:
```
start:
> topic(text(20)): the general category from which the question was taken
> question(sentence<50,english>): the question that you have to answer
> choices[4]: you judge whether each choice is correct or not
> > value(sentence<40,english>): the value of the choice
> > correct(bool): you decide whether this choice is correct or not
> answer(response): you repeat the value of the choice that best answer the question
```
It includes the folowing named formats:
- sentence<50,english>: text(50)
  - a grammatically correct english sentence of at most 50 tokens
- sentence<40,english>: text(40)
  - a grammatically correct english sentence of at most 40 tokens
- bool: enum("true","false")
  - a bolean value is either true or false
- response: repeat(choices.value)
  - repeat verbatim the value of the correct choice
n_header [label="[INST]\n<<SYS>>You are an expert interacting with t